# Davidson-MacKinnon J-Test Tutorial: Comparing Non-Nested Models

This tutorial demonstrates how to use the Davidson-MacKinnon J-test to compare non-nested model specifications in panel data.

## Introduction

The J-test is used when you have two competing models that are **not nested** (i.e., neither model is a special case of the other). Common examples include:

- Linear vs. log-linear specifications
- Different functional forms (Cobb-Douglas vs. Translog production functions)
- Different sets of explanatory variables

### How the J-Test Works

Given two models:
- **Model 1**: $y = X_1'\beta_1 + \varepsilon_1$
- **Model 2**: $y = X_2'\beta_2 + \varepsilon_2$

The J-test:
1. Estimates both models and obtains fitted values $\hat{y}_1$ and $\hat{y}_2$
2. **Forward test**: Augments Model 1 with $\hat{y}_2$ and tests if $\alpha=0$ in:
   $$y = X_1'\beta_1 + \alpha \hat{y}_2 + \varepsilon$$
3. **Reverse test**: Augments Model 2 with $\hat{y}_1$ and tests if $\gamma=0$ in:
   $$y = X_2'\beta_2 + \gamma \hat{y}_1 + \varepsilon$$

### Interpretation

| Forward Test | Reverse Test | Interpretation |
|--------------|--------------|----------------|
| Reject H₀    | Don't reject | Prefer Model 2 |
| Don't reject | Reject H₀    | Prefer Model 1 |
| Reject both  | Reject both  | Neither model adequate |
| Don't reject | Don't reject | Both models acceptable |

In [ ]:
import numpy as np
import pandas as pd
import panelbox as pb
from panelbox.diagnostics.specification import j_test
import matplotlib.pyplot as plt
import seaborn as sns

# Set random seed for reproducibility
np.random.seed(42)
sns.set_style('whitegrid')

## Example 1: Production Function Specification

We'll compare two production function specifications:
- **Model 1 (Cobb-Douglas)**: $\log(Y) = \beta_0 + \beta_1 \log(K) + \beta_2 \log(L) + \varepsilon$
- **Model 2 (Translog)**: Includes interaction and squared terms

### Generate Simulated Data

In [ ]:
# Generate panel data for firms
n_firms = 100
n_years = 10
n_obs = n_firms * n_years

# Create panel structure
firms = np.repeat(np.arange(n_firms), n_years)
years = np.tile(np.arange(n_years), n_firms)

# Generate inputs (capital and labor)
log_K = np.random.randn(n_obs) + 5  # Log capital
log_L = np.random.randn(n_obs) + 4  # Log labor

# True DGP: Cobb-Douglas with some nonlinearity
log_Y_true = 2.0 + 0.3 * log_K + 0.7 * log_L + 0.05 * log_K * log_L
log_Y = log_Y_true + np.random.randn(n_obs) * 0.3

# Create DataFrame
df = pd.DataFrame({
    'firm': firms,
    'year': years,
    'log_Y': log_Y,
    'log_K': log_K,
    'log_L': log_L,
    'log_K_sq': log_K**2,
    'log_L_sq': log_L**2,
    'log_K_log_L': log_K * log_L
})

print("Data shape:", df.shape)
print("\nFirst few observations:")
print(df.head(10))

### Estimate Model 1: Cobb-Douglas

In [ ]:
# Model 1: Simple Cobb-Douglas
model1 = pb.PooledOLS(
    "log_Y ~ log_K + log_L",
    data=df,
    entity_col='firm',
    time_col='year'
)
result1 = model1.fit(cov_type='clustered')
print("Model 1: Cobb-Douglas")
print(result1.summary())

### Estimate Model 2: Translog (with Interaction)

In [ ]:
# Model 2: Translog with interaction
model2 = pb.PooledOLS(
    "log_Y ~ log_K + log_L + log_K_log_L",
    data=df,
    entity_col='firm',
    time_col='year'
)
result2 = model2.fit(cov_type='clustered')
print("Model 2: Translog (with interaction)")
print(result2.summary())

### Perform J-Test

In [ ]:
# Perform J-test
jtest_result = j_test(result1, result2, direction='both')

print("\n" + "="*70)
print("Davidson-MacKinnon J-Test Results")
print("="*70)
print(jtest_result.summary())

# Show interpretation
print("\n" + "="*70)
print("INTERPRETATION")
print("="*70)
print(jtest_result.interpretation())

### Visualize Model Comparison

In [ ]:
# Get fitted values
fitted1 = result1.predict()
fitted2 = result2.predict()

# Create comparison plot
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Plot 1: Actual vs Fitted (Model 1)
axes[0].scatter(df['log_Y'], fitted1, alpha=0.5, s=20)
axes[0].plot([df['log_Y'].min(), df['log_Y'].max()], 
             [df['log_Y'].min(), df['log_Y'].max()], 
             'r--', lw=2, label='45° line')
axes[0].set_xlabel('Actual log(Y)')
axes[0].set_ylabel('Fitted log(Y)')
axes[0].set_title(f'Model 1: Cobb-Douglas\nR² = {result1.rsquared:.4f}')
axes[0].legend()
axes[0].grid(alpha=0.3)

# Plot 2: Actual vs Fitted (Model 2)
axes[1].scatter(df['log_Y'], fitted2, alpha=0.5, s=20, color='orange')
axes[1].plot([df['log_Y'].min(), df['log_Y'].max()], 
             [df['log_Y'].min(), df['log_Y'].max()], 
             'r--', lw=2, label='45° line')
axes[1].set_xlabel('Actual log(Y)')
axes[1].set_ylabel('Fitted log(Y)')
axes[1].set_title(f'Model 2: Translog\nR² = {result2.rsquared:.4f}')
axes[1].legend()
axes[1].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## Example 2: Linear vs. Log-Linear Specification

Compare:
- **Model 3 (Linear)**: $Y = \beta_0 + \beta_1 X_1 + \beta_2 X_2 + \varepsilon$
- **Model 4 (Log-Linear)**: $\log(Y) = \beta_0 + \beta_1 X_1 + \beta_2 X_2 + \varepsilon$

In [ ]:
# Generate new data for this example
X1 = np.random.randn(n_obs)
X2 = np.random.randn(n_obs)

# True DGP: exponential relationship
Y_level = np.exp(0.5 + 0.3 * X1 + 0.4 * X2 + np.random.randn(n_obs) * 0.2)
log_Y_new = np.log(Y_level)

df2 = pd.DataFrame({
    'firm': firms,
    'year': years,
    'Y': Y_level,
    'log_Y': log_Y_new,
    'X1': X1,
    'X2': X2
})

# Model 3: Linear
model3 = pb.PooledOLS("Y ~ X1 + X2", data=df2, entity_col='firm', time_col='year')
result3 = model3.fit(cov_type='clustered')

# Model 4: Log-linear
model4 = pb.PooledOLS("log_Y ~ X1 + X2", data=df2, entity_col='firm', time_col='year')
result4 = model4.fit(cov_type='clustered')

print("Model 3: Linear")
print(result3.summary())
print("\nModel 4: Log-Linear")
print(result4.summary())

In [ ]:
# Note: For J-test, we need models with same dependent variable
# We'll use log-transformed fitted values from Model 3
print("NOTE: When comparing linear vs. log-linear, ensure both models")
print("      use the same dependent variable. Here we show the conceptual")
print("      approach - in practice, you'd transform predictions appropriately.")

# For illustration, test linear vs. a different linear specification
model3b = pb.PooledOLS("Y ~ X1", data=df2, entity_col='firm', time_col='year')
result3b = model3b.fit(cov_type='clustered')

jtest_result2 = j_test(result3, result3b, direction='both')
print("\nJ-Test: Full Model vs. Restricted Model")
print(jtest_result2.summary())
print("\nInterpretation:")
print(jtest_result2.interpretation())

## Summary and Best Practices

### When to Use J-Test
1. **Non-nested models**: Neither model is a special case of the other
2. **Same dependent variable**: Both models must explain the same outcome
3. **Theory-driven comparison**: Economic theory suggests multiple specifications

### Interpreting Results
- **Both reject**: Neither model adequate → consider alternative specifications
- **Both don't reject**: Both acceptable → use economic theory, simplicity, or other criteria
- **One rejects**: Clear preference for one model

### Limitations
- Low power with small samples
- Can be sensitive to outliers
- Should be combined with other model selection criteria (AIC, BIC, economic theory)

### Next Steps
- Combine with encompassing tests for more robust conclusions
- Use cross-validation for out-of-sample performance
- Consider economic interpretation and theoretical foundations